### Funciones requeridas para la API

Desarrollaremos y probaremos las funciones que han sido solicitadas para la API. Luego, cuando tengamos que deployarlas, es posible que debamos realizar algunos cambios debido a los requerimientos propios de FastAPI o Render.

### Carga de los dataset
Cargaremos los conjuntos de datos necesarios para llevar a cabo las consultas solicitadas. Decidimos subir los datos de esta manera, y no los dataset completos, debido a la limitada capacidad de almacenamiento que permite Render. Por otra parte, considerando que el objetivo es entregar un PVM, a modo de prueba de concepto, considero adecuada esta simplificación de los datasets.

Los dataset que se utilizan son:

* df_user_reviews_FE: contiene la información relacionada a los usuarios que realizan reviews de los juegos en formato de análisis de sentimientos. Entre esta información, se encuentran las recomendaciones o no del juego por parte de usuario.

* df_user_data_FE: contiene la cantidad de items que consume cada usuario y el precio de cada uno de los productos que consume.

* df_userforgenre_FE: contiene el id de cada usuario, la cantidad de horas jugadas por cada género de juego y el año de lanzamiento.

* df_best_developer_FE: contiene el ranking de los géneros de juegos con mas horas jugadas.

* df_item_developer_year_FE: contiene la información relacionada con cada item de juegos como su id, desarrollador, las recomendaciones y el analisis de sentimiento.



In [1]:
import pandas as pd
import tools
import warnings
warnings.filterwarnings("ignore")


df_user_reviews = pd.read_csv("df_user_reviews_FE.csv")
df_user_data = pd.read_csv("df_user_data_FE.csv")
df_user_for_genre = pd.read_csv("df_userforgenre_FE.csv")
df_item_developer_year = pd.read_csv("df_item_developer_year_FE.csv")
df_best_developer = pd.read_csv("df_best_developer_FE.csv")


A continuación, se desarrollan cada una de las funciones solicitadas.
# developer
Esta función recibe como parámetro 'developer', que es la empresa desarrolladora del juego, y devuelve la cantidad de items que desarrolla dicha empresa y el porcentaje de contenido gratuito por año por sobre el total que desarrolla

In [18]:
def developer(desarrollador: str):
    # Filtramos el dataframe por desarrollador de interés
    data_filtrada = df_item_developer_year[df_item_developer_year["developer"] == desarrollador]
    # La cantidad de items por año
    cantidad_por_año = data_filtrada.groupby("release_year")["item_id"].count()
    # La cantidad de elementos gratis por año
    cantidad_gratis_por_año = data_filtrada[data_filtrada["price"] == 0.0].groupby("release_year")["item_id"].count()
    # El porcentaje de elementos gratis por año
    porcentaje_gratis_por_año = (cantidad_gratis_por_año / cantidad_por_año * 100).fillna(0).astype(int)
    # Agregamos el símbolo de porcentaje (%) al valor del porcentaje
    porcentaje_gratis_por_año = porcentaje_gratis_por_año.astype(str) + '%'
    
    result_dict = {
        "cantidad_por_año": cantidad_por_año.to_dict(),
        "porcentaje_gratis_por_año": porcentaje_gratis_por_año.to_dict()
    }
    
    return result_dict

In [19]:
desarrollador = "Kotoshiro"
developer(desarrollador)

{'cantidad_por_año': {'2018': 1}, 'porcentaje_gratis_por_año': {'2018': 0}}

## userdata
Esta función tiene por parámentro 'user_id' y devuelve la cantidad de dinero gastado por el usuario, el porcentaje de recomendaciones que realizó sobre la cantidad de reviews que se analizan y la cantidad de items que consume el mismo.

In [20]:
def userdata(user_id):
    
    # Filtramos por el usuario de interés
    user= df_user_reviews[df_user_reviews["user_id"] == user_id]
    # la cantidad de dinero gastado para el usuario de interés
    cantidad = df_user_data[df_user_data["user_id"]== user_id]["price"].iloc[0]
    # Buscamos el count_item para el usuario de interés    
    conteo_items = df_user_data[df_user_data["user_id"]== user_id]["items_count"].iloc[0]
    
    # Total de recomendaciones realizadas por el usuario de interés
    total_recomendaciones = user["reviews_recommend"].sum()
    # Total de reviews realizada por todos los usuarios
    total_reviews = len(df_user_reviews["user_id"].unique())
    # Porcentaje de recomendaciones realizadas por el usuario de interés
    porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100
    
    return {
        "cantidad_dinero": cantidad,
        "porcentaje_recomendacion": round(porcentaje_recomendaciones, 2),
        "total_items": conteo_items.astype(int)
    }

In [21]:
user_id = "ButtBurger2"
userdata(user_id)

{'cantidad_dinero': 1071.15,
 'porcentaje_recomendacion': 0.03,
 'total_items': 116}

### UserForGenre
Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.


In [22]:
def UserForGenre(genero:str):
    
    # Filtramos el DataFrame por el género dado
    genre_data = df_user_for_genre[df_user_for_genre["genres"] == genero]

    # Usuario con más horas jugadas para ese género
    top_user = genre_data.loc[genre_data["played_hours"].idxmax()]["user_id"]

    # Lista de acumulación de horas jugadas por año
    hours_by_year = genre_data.groupby("release_year")["played_hours"].sum().reset_index()
  
    hours_by_year = hours_by_year.rename(columns={"release_year": "Año", "played_hours": "Horas"})
    
    hours_list = hours_by_year.to_dict(orient="records")

    # Diccionario de retorno
    result = {
        "Usuario con más horas jugadas para Género {}".format(genero): top_user,
        "Horas jugadas": hours_list
    }

    return result

In [23]:
genero = "Indie"
resultado = UserForGenre(genero)
print(resultado)

{'Usuario con más horas jugadas para Género Indie': 'REBAS_AS_F-T', 'Horas jugadas': [{'Año': 2005, 'Horas': 5511.23}, {'Año': 2006, 'Horas': 7435258.0}, {'Año': 2007, 'Horas': 22336.16}, {'Año': 2008, 'Horas': 87594.71}, {'Año': 2009, 'Horas': 63460.65}, {'Año': 2010, 'Horas': 142221.89}, {'Año': 2011, 'Horas': 3756228.85}, {'Año': 2012, 'Horas': 2052902.27}, {'Año': 2013, 'Horas': 2955802.75}, {'Año': 2014, 'Horas': 1536246.04}, {'Año': 2015, 'Horas': 499837.09}, {'Año': '1988', 'Horas': 232.76}, {'Año': '1995', 'Horas': 52.839999999999996}, {'Año': '1996', 'Horas': 9.53}, {'Año': '1997', 'Horas': 1071.43}, {'Año': '1998', 'Horas': 332.65}, {'Año': '1999', 'Horas': 10450.29}, {'Año': '2000', 'Horas': 88.4}, {'Año': '2001', 'Horas': 1474.24}, {'Año': '2002', 'Horas': 631.53}, {'Año': '2003', 'Horas': 35456.25}, {'Año': '2004', 'Horas': 719.58}, {'Año': '2005', 'Horas': 93.95}, {'Año': '2015', 'Horas': 2393073.9}, {'Año': '2016', 'Horas': 1488879.61}, {'Año': '2017', 'Horas': 2093241.1

### best_developer_year: 

Devuelve el top 3 de desarrolladores con juegos más recomendados por usuarios para el año dado. 

In [6]:
def best_developer_year(anio):
    
    # Filtramos por el año dado y solo con comentarios recomendados y positivos
    df_filtrado = df_best_developer[(df_best_developer["year"] == anio) & (df_best_developer["reviews_recommend"] == True) & (df_best_developer["sentiment_analysis"] == 2)]

    if df_filtrado.empty:
        return f"No hay datos para el año {anio}."

    # Agrupamos por desarrollador y contamos la cantidad de juegos recomendados
    desarrolladores_top = df_filtrado.groupby("developer")["item_id"].count().reset_index()

    # Ordenamos en orden descendente y seleccionar los tres primeros
    desarrolladores_top = desarrolladores_top.sort_values(by="item_id", ascending=False).head(3)

    # Creamos el resultado en el formato deseado
    resultado = [{"Puesto " + str(i+1): desarrollador} for i, desarrollador in enumerate(desarrolladores_top['developer'])]

    return resultado



In [7]:
anio_deseado = 2015
resultado = best_developer_year(anio_deseado)
print(resultado)

[{'Puesto 1': 'Valve'}, {'Puesto 2': 'Facepunch Studios'}, {'Puesto 3': 'Smartly Dressed Games'}]


### developer_reviews_analysis

Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [3]:
def developer_reviews_analysis(desarrolladora):
    # Supongamos que tienes un DataFrame llamado 'df' con las columnas proporcionadas
    
    # Filtrar por desarrolladora
    df_filtrado = df_best_developer[df_best_developer["developer"] == desarrolladora]

    if df_filtrado.empty:
        return f"No hay datos para la desarrolladora {desarrolladora}."

    # Contar la cantidad de registros con análisis de sentimiento 0, 1 y 2
    conteo_sentimientos = df_filtrado["sentiment_analysis"].value_counts()

    # Crear el resultado en el formato deseado
    resultado = {desarrolladora: {"Negative": conteo_sentimientos.get(0, 0),
                                  "Positive": conteo_sentimientos.get(2, 0)}}

    return resultado


In [4]:
# Ejemplo de uso
desarrolladora_deseada = "Smartly Dressed Games"
resultado = developer_reviews_analysis(desarrolladora_deseada)
print(resultado)

{'Smartly Dressed Games': {'Negative': 395, 'Positive': 1470}}
